# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [7]:
!pip install -qU ragas==0.2.10


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 0.7.700 requires aiofiles<24.0.0,>=23.1.0, but you have aiofiles 24.1.0 which is incompatible.
chainlit 0.7.700 requires httpx<0.25.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [3]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: data: File exists


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31427    0 31427    0     0   143k      0 --:--:-- --:--:-- --:--:--  144k


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70286    0 70286    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
import nltk

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/christinemahler/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/christinemahler/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [8]:
from langchain_community.document_loaders import DirectoryLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,what happen in 2023 with LLMs?,[Code may be the best application The ethics o...,"In 2023, there were significant advancements i...",single_hop_specifc_query_synthesizer
1,How do Large Language Models (LLMs) handle the...,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What significant advancements in Large Languag...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Large Language Models (LLMs) experien...",single_hop_specifc_query_synthesizer
3,Wht are the implicashuns of the leaked Googel ...,[easy to follow. The rest of the document incl...,The leaked Google document titled 'We Have No ...,single_hop_specifc_query_synthesizer
4,How do Large Language Models (LLMs) balance th...,[<1-hop>\n\nreasoning patterns. Another common...,Large Language Models (LLMs) are powerful tool...,multi_hop_abstract_query_synthesizer
5,How have advancements in LLM efficiency impact...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in LLM efficiency have significan...,multi_hop_abstract_query_synthesizer
6,How do the advancements in Large Language Mode...,[<1-hop>\n\nreasoning patterns. Another common...,The advancements in Large Language Models (LLM...,multi_hop_abstract_query_synthesizer
7,How have advancements in large language models...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in large language models (LLMs) h...,multi_hop_abstract_query_synthesizer
8,How does the gullibility of ChatGPT impact its...,[<1-hop>\n\nBased Development As a computer sc...,The gullibility of ChatGPT significantly impac...,multi_hop_specific_query_synthesizer
9,How has the introduction of GPT-4o impacted th...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The introduction of GPT-4o has significantly i...,multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [12]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/93ff5d50-b1e1-4afc-85ca-39c8dee1792b


'https://app.ragas.io/dashboard/alignment/testset/93ff5d50-b1e1-4afc-85ca-39c8dee1792b'

![title](<RAGAS Test Dataset.png>)

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [13]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

73

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

**A chunk overlap has to preserve meaning between chunks. By overlapping the chunks, we can potentially avoid losing important content at either end of the chunk.**

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [15]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [16]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [17]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [18]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [19]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [20]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [22]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [23]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [24]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [25]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [26]:
response["response"]

'LLM (Large Language Model) agents can be useful in several ways, primarily through their ability to act on behalf of users and solve problems with the assistance of tools. The context highlights two main perspectives on their utility: \n\n1. **Acting on Behalf of Users**: Some view LLM agents as digital assistants that can perform tasks similar to a travel agent, making decisions and acting autonomously to benefit users.\n\n2. **Problem Solving**: Others see their value in running loops with tools to solve problems effectively. This can include generating code and iterating on it until it works, which demonstrates a practical application of LLMs in software development.\n\nDespite their potential, there is skepticism about their reliability due to the tendency of LLMs to "hallucinate" or generate false information. This raises concerns about how effective they can be in making meaningful decisions without being able to distinguish truth from fiction. \n\nOverall, while LLM agents have

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [27]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [28]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,what happen in 2023 with LLMs?,[This is Things we learned about LLMs in 2024 ...,[Code may be the best application The ethics o...,"In 2023, significant advancements were made in...","In 2023, there were significant advancements i...",single_hop_specifc_query_synthesizer
1,How do Large Language Models (LLMs) handle the...,[Code may be the best application\n\nThe ethic...,[Based Development As a computer scientist and...,Large Language Models (LLMs) handle the gramma...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What significant advancements in Large Languag...,[Simon Willison’s Weblog\n\nSubscribe\n\nStuff...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, significant advancements in Large Lan...","In 2023, Large Language Models (LLMs) experien...",single_hop_specifc_query_synthesizer
3,Wht are the implicashuns of the leaked Googel ...,[Article Visitors Pageviews Bing: “I will not ...,[easy to follow. The rest of the document incl...,"The leaked Google document titled ""We Have No ...",The leaked Google document titled 'We Have No ...,single_hop_specifc_query_synthesizer
4,How do Large Language Models (LLMs) balance th...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nreasoning patterns. Another common...,Large Language Models (LLMs) face significant ...,Large Language Models (LLMs) are powerful tool...,multi_hop_abstract_query_synthesizer
5,How have advancements in LLM efficiency impact...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in the efficiency of large langua...,Advancements in LLM efficiency have significan...,multi_hop_abstract_query_synthesizer
6,How do the advancements in Large Language Mode...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nreasoning patterns. Another common...,The advancements in Large Language Models (LLM...,The advancements in Large Language Models (LLM...,multi_hop_abstract_query_synthesizer
7,How have advancements in large language models...,[Another common technique is to use larger mod...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in large language models (LLMs) h...,Advancements in large language models (LLMs) h...,multi_hop_abstract_query_synthesizer
8,How does the gullibility of ChatGPT impact its...,[Gullibility is the biggest unsolved problem\n...,[<1-hop>\n\nBased Development As a computer sc...,The gullibility of ChatGPT significantly impac...,The gullibility of ChatGPT significantly impac...,multi_hop_specific_query_synthesizer
9,How has the introduction of GPT-4o impacted th...,[Today $30/mTok gets you OpenAI’s most expensi...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The introduction of GPT-4o has significantly i...,The introduction of GPT-4o has significantly i...,multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [29]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [30]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [31]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[11]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 29751, Requested 2108. Please try again in 3.718s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 29413, Requested 2447. Please try again in 3.72s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[19]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 28946, Requested 

{'context_recall': 0.6429, 'faithfulness': 0.8400, 'factual_correctness': 0.4183, 'answer_relevancy': 0.9498, 'context_entity_recall': 0.3316, 'noise_sensitivity_relevant': 0.3842}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [32]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

In [34]:
#!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [33]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [34]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [35]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [36]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents are considered useful primarily in two ways: \n\n1. **Writing Code**: One of the most notable capabilities of LLMs is their effectiveness in writing code. They can handle the grammar rules of programming languages, which are less complicated than natural languages, making them particularly adept at this task.\n\n2. **Acting on Behalf of Users**: There is excitement around the potential for AI agents to act autonomously on behalf of users, similar to a travel agent. This concept encompasses AI systems that can take actions without constant human oversight.\n\nHowever, there are significant challenges to their utility, particularly the issue of "gullibility." LLMs can struggle to distinguish truth from fiction, which raises concerns about their ability to make meaningful decisions. This skepticism towards their utility also leads to a cautious approach in discussing their applications, as the technology is still evolving and there are few real-world examples of LLM agents in 

In [37]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [38]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[20]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 29647, Requested 1851. Please try again in 2.996s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 29765, Requested 2447. Please try again in 4.424s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[19]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-fG4XzykR5yuSt2Wp6ZlK7gAJ on tokens per min (TPM): Limit 30000, Used 29254, Requested

{'context_recall': 0.6111, 'faithfulness': 0.7500, 'factual_correctness': 0.4464, 'answer_relevancy': 0.9495, 'context_entity_recall': 0.3548, 'noise_sensitivity_relevant': 0.3269}

#### ❓ Question: 

Which system performed better, on what metrics, and why?

**Context recall went from .6429 to .6111 (degraded), faithfulness from .84 to .75 (degraded), factual correctness from .4183 to .4464 (improved), answer relevancy from .9498 to .9495 (only slightly degraded), context entity recall from .3316 to .3548 (improved), and noise sensitivity from .3842 to .3269 (improved). If the objective of using Cohere rerank was to improve the accuracy of the LLM output, then I would say we achieved it albeit at the cost of degrading the context recall and faithfulness metrics. If, however, we were trying to stay true to our context (notwithstanding why exactly we would want to do that), the 2nd run would reflect a worse performance.**